In [1]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts 
from api_keys import air_pollution_api_key
import os

In [2]:
base_url = 'https://api.waqi.info/feed/?'
print(base_url)
print(requests.get(base_url))

https://api.waqi.info/feed/?
<Response [200]>


In [4]:
path = "air_pollution_2020.csv"
air_pollution = pd.read_csv(path)
air_pollution

,Date,Country,City,Specie,count,min,max,median,variance
0,1/7/20,CA,Hamilton,wind speed,59,0.1,3.0,1.0,5.41
1,1/31/20,CA,Hamilton,wind speed,13,0.1,0.3,0.1,0.03
2,1/5/20,CA,Hamilton,wind speed,78,0.1,4.6,0.7,13.41
3,1/21/20,CA,Hamilton,wind speed,76,0.1,3.2,1.2,8.22
4,1/23/20,CA,Hamilton,wind speed,12,0.1,0.1,0.1,0.00
...,...,...,...,...,...,...,...,...,...
595787,1/11/20,TJ,Dushanbe,pressure,24,1010.0,1018.0,1014.5,85.14
595788,3/15/20,TJ,Dushanbe,pressure,24,1015.0,1021.0,1018.0,35.87
595789,3/16/20,TJ,Dushanbe,pressure,24,1016.5,1019.0,1018.0,6.36
595790,3/21/20,TJ,Dushanbe,pressure,24,1014.0,1022.0,1018.0,71.30


In [5]:
air_pollution["Specie"].value_counts()

pm25             54185
temperature      53220
humidity         53212
pressure         53048
pm10             52784
no2              52567
o3               48741
wind speed       44075
so2              43905
dew              40614
co               40488
wind gust        28267
wind-speed       11749
wind-gust         6572
precipitation     4283
wd                4009
aqi               1539
uvi               1075
pol                715
pm1                421
neph               216
mepaqi             107
Name: Specie, dtype: int64

In [6]:
new_air_pollution = air_pollution[["Date","Country", "City", "Specie", "median"]]
new_air_pollution

,Date,Country,City,Specie,median
0,1/7/20,CA,Hamilton,wind speed,1.0
1,1/31/20,CA,Hamilton,wind speed,0.1
2,1/5/20,CA,Hamilton,wind speed,0.7
3,1/21/20,CA,Hamilton,wind speed,1.2
4,1/23/20,CA,Hamilton,wind speed,0.1
...,...,...,...,...,...
595787,1/11/20,TJ,Dushanbe,pressure,1014.5
595788,3/15/20,TJ,Dushanbe,pressure,1018.0
595789,3/16/20,TJ,Dushanbe,pressure,1018.0
595790,3/21/20,TJ,Dushanbe,pressure,1018.0


In [8]:
cal_aqi = new_air_pollution.loc[(new_air_pollution["Specie"] == "pm1") |\
                        (new_air_pollution["Specie"] == "pm1") |\
                        (new_air_pollution["Specie"] == "o3") |\
                        (new_air_pollution["Specie"] == "no2") |\
                        (new_air_pollution["Specie"] == "so2") |\
                        (new_air_pollution["Specie"] == "co") |\
                        (new_air_pollution["Specie"] == "pm25"), :]

cal_aqi.count()

Date       240307
Country    240307
City       240307
Specie     240307
median     240307
dtype: int64

In [9]:
cal_aqi.head()

,Date,Country,City,Specie,median
164,2/29/20,CA,Hamilton,co,1.9
165,3/3/20,CA,Hamilton,co,3.9
166,1/5/20,CA,Hamilton,co,1.7
167,3/9/20,CA,Hamilton,co,2.3
168,3/7/20,CA,Hamilton,co,1.9


In [10]:
#Checking how many datapoints for US
cal_aqi = cal_aqi.loc[(cal_aqi["Country"] == "US")]


In [11]:
aqi_city_2020 = round(cal_aqi.groupby("City").mean(), 2)
aqi_city_2020.head()

,median
City,
Albuquerque,11.35
Atlanta,14.44
Austin,24.70
Baltimore,18.44
Boise,12.49


In [12]:
aqi_city = aqi_city_2020.rename(columns={'median': 'aqi'})
aqi_city.head()

,aqi
City,
Albuquerque,11.35
Atlanta,14.44
Austin,24.70
Baltimore,18.44
Boise,12.49


In [13]:
#write to csv
pd.DataFrame.to_csv(aqi_city, 'aqi_city.csv')

In [14]:
path_1 = "zip_code_database.csv"
cities_counties = pd.read_csv(path_1)
cities_counties = cities_counties[["primary_city", "county", "state"]]
cities_counties = cities_counties.rename(columns={'primary_city': "City"})
cities_counties = cities_counties.drop_duplicates(["City", "county"], keep= 'last')
cities_counties = cities_counties.dropna()


In [15]:
#write to csv
pd.DataFrame.to_csv(cities_counties, 'cities_counties.csv')

In [16]:
#read csv
path_cities_counties = "cities_counties.csv"
cities_counties_df = pd.read_csv(path_cities_counties)
cities_counties_df

,Unnamed: 0,City,county,state
0,2,Adjuntas,Adjuntas Municipio,PR
1,3,Aguada,Aguada Municipio,PR
2,4,Aguadilla,Aguadilla Municipio,PR
3,7,Maricao,Maricao Municipio,PR
4,8,Anasco,Anasco Municipio,PR
...,...,...,...,...
29813,42627,Metlakatla,Prince of Wales-Outer Ketchikan Borough,AK
29814,42628,Point Baker,Prince of Wales-Hyder Census Area,AK
29815,42629,Ward Cove,Ketchikan Gateway Borough,AK
29816,42630,Wrangell,Wrangell City and Borough,AK


In [17]:
merge = cities_counties.merge(aqi_city, on='City')
merge

,City,county,state,aqi
0,San Antonio,Aguadilla Municipio,PR,26.28
1,San Antonio,Pasco County,FL,26.28
2,San Antonio,Comal County,TX,26.28
3,San Antonio,Bexar County,TX,26.28
4,San Antonio,Socorro County,NM,26.28
...,...,...,...,...
392,Los Angeles,Los Angeles County,CA,19.91
393,San Francisco,San Mateo County,CA,12.89
394,San Francisco,San Francisco County,CA,12.89
395,Honolulu,Honolulu County,HI,15.64


In [18]:
path_2 = "NYT_counties_data.csv"
counties_data = pd.read_csv(path_2)
counties_data = counties_data[["date", "county", "cases", "deaths", "state"]]
counties_data = counties_data.loc[(counties_data["date"] == "2020-04-10")]


In [19]:
#write to csv
pd.DataFrame.to_csv(counties_data, 'counties_data.csv')

In [20]:
#read csv
path_counties = "counties_data.csv"
counties_df = pd.read_csv(path_counties)
counties_df

,Unnamed: 0,date,county,cases,deaths,state
0,45880,2020-04-10,Autauga,17,1,Alabama
1,45881,2020-04-10,Baldwin,59,1,Alabama
2,45882,2020-04-10,Barbour,9,0,Alabama
3,45883,2020-04-10,Bibb,11,0,Alabama
4,45884,2020-04-10,Blount,12,0,Alabama
...,...,...,...,...,...,...
2624,48504,2020-04-10,Sublette,1,0,Wyoming
2625,48505,2020-04-10,Sweetwater,6,0,Wyoming
2626,48506,2020-04-10,Teton,50,0,Wyoming
2627,48507,2020-04-10,Uinta,4,0,Wyoming


In [24]:
merged = cities_counties.merge(counties_df, on=['county', 'state'], how="outer")
merged

,City,county,state,Unnamed: 0,date,cases,deaths
0,Adjuntas,Adjuntas Municipio,PR,NaN,NaN,NaN,NaN
1,Aguada,Aguada Municipio,PR,NaN,NaN,NaN,NaN
2,Aguadilla,Aguadilla Municipio,PR,NaN,NaN,NaN,NaN
3,San Antonio,Aguadilla Municipio,PR,NaN,NaN,NaN,NaN
4,Maricao,Maricao Municipio,PR,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
32442,NaN,Sublette,Wyoming,48504.0,2020-04-10,1.0,0.0
32443,NaN,Sweetwater,Wyoming,48505.0,2020-04-10,6.0,0.0
32444,NaN,Teton,Wyoming,48506.0,2020-04-10,50.0,0.0
32445,NaN,Uinta,Wyoming,48507.0,2020-04-10,4.0,0.0


In [81]:
groupby_city = merged.groupby("City").mean()
groupby_city

,latitude,longitude,cases,deaths
City,,,,
Akiachak,60.900000,-161.420000,1.000000,0.000000
Akiak,60.900000,-161.230000,1.000000,0.000000
Alexandria,38.821667,-77.083333,37.666667,0.133333
Allakaket,66.570000,-152.950000,1.000000,0.000000
Anchor Point,59.820000,-151.590000,7.166667,0.083333
...,...,...,...,...
Wasilla,61.486667,-149.693333,3.578947,0.000000
Waynesboro,38.060000,-78.900000,2.545455,0.000000
Williamsburg,37.270000,-76.720000,7.760000,0.800000


In [77]:
merged_2 = merged.merge(aqi_city, on='City')
merged_2

,City,county,state,latitude,longitude,date,cases,deaths,aqi
0,Baltimore,Baltimore city,MD,39.29,-76.62,2020-03-15,1,0,18.44
1,Baltimore,Baltimore city,MD,39.29,-76.62,2020-03-16,1,0,18.44
2,Baltimore,Baltimore city,MD,39.29,-76.62,2020-03-17,3,0,18.44
3,Baltimore,Baltimore city,MD,39.29,-76.62,2020-03-18,7,0,18.44
4,Baltimore,Baltimore city,MD,39.29,-76.62,2020-03-19,7,0,18.44
...,...,...,...,...,...,...,...,...,...
758,Houston,Matanuska-Susitna Borough,AK,61.62,-149.79,2020-04-06,4,0,25.67
759,Houston,Matanuska-Susitna Borough,AK,61.62,-149.79,2020-04-07,4,0,25.67
760,Houston,Matanuska-Susitna Borough,AK,61.62,-149.79,2020-04-08,5,0,25.67
761,Houston,Matanuska-Susitna Borough,AK,61.62,-149.79,2020-04-09,6,0,25.67
